In [3]:
!pip install openai
!pip install langchain
!pip install eodhd
!pip install config

INFO: pip is looking at multiple versions of eodhd to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of eodhd to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/8.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/8.5 MB 17.1 MB/s eta 0:00:01
   ------- -------------------------------- 1.5/8.5 MB 13.8 MB/s eta 0:00:01
   --------- ------------------------------ 2.1/8.5 MB 13.1 MB/s eta 0:00:01
   ------------ --------------------------- 2.6/8.5 MB 12.8 MB/s eta 0:00:01
   -------------- ------------------------- 3.2/8.5 MB 12.6 MB/s eta 0:00:01
   ------------------ --------------------- 3.9/8.5 MB 12.4 MB/s eta 0:00:01
   -------------------- ------------------- 4.4/8.5 MB 12.2 MB/s eta 0:00:01
   ----------------------- ---------------- 5.0/8.5 MB 12.3 MB/s eta 0:00:01
   ----------------------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mxnet 1.7.0.post2 requires numpy<1.17.0,>=1.8.2, but you have numpy 1.21.6 which is incompatible.
mxnet 1.7.0.post2 requires requests<2.19.0,>=2.18.4, but you have requests 2.28.1 which is incompatible.
chromadb 0.4.21 requires numpy>=1.22.5, but you have numpy 1.21.6 which is incompatible.
chromadb 0.4.21 requires tokenizers>=0.13.2, but you have tokenizers 0.10.3 which is incompatible.
seaborn 0.13.2 requires matplotlib!=3.6.1,>=3.4, but you have matplotlib 3.3.3 which is incompatible.
unstructured-client 0.15.1 requires charset-normalizer>=3.2.0, but you have charset-normalizer 2.0.4 which is incompatible.
unstructured-client 0.15.1 requires requests>=2.31.0, but you have requests 2.28.1 which is incom

In [53]:
import re
import requests
import pandas as pd
from eodhd import APIClient # EOD Historical Data로 data API 제공
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [54]:
api_key = ''
api = APIClient(api_key)

In [55]:
resp = api.financial_news(s = "MSFT", from_date = '2024-01-01', to_date = '2024-03-20', limit = 100)
df = pd.DataFrame(resp) # json 출력을 datframe으로 변환
df.tail()

,date,title,content,link,symbols,tags,sentiment
95,2024-03-14T00:00:00+00:00,7 Delightful Dividend Stocks to Put Some Sprin...,The best dividend stocks can be counted on to ...,https://investorplace.com/market360/2024/03/7-...,"[AAPL.US, AVGO.US, JPM.US, LLY.US, MA.US, MSFT...",[],"{'polarity': 0.856, 'neg': 0, 'neu': 0.732, 'p..."
96,2024-03-14T00:00:00+00:00,3 Virtual Reality Stocks to Buy as the VR Mark...,While you might not personally be a fan of dig...,https://investorplace.com/2024/03/3-virtual-re...,"[META.US, MSFT.US, U.US]",[],"{'polarity': 0.399, 'neg': 0.081, 'neu': 0.792..."
97,2024-03-14T00:00:00+00:00,"\nDanny Vena | Mar 14, 2024\n",Wall Street sees big upside for the company's ...,https://www.fool.com/investing/2024/03/14/why-...,[MSFT.US],[],"{'polarity': 0.477, 'neg': 0, 'neu': 0.78, 'po..."
98,2024-03-14T00:00:00+00:00,"\nTrevor Jennewine | Mar 14, 2024\n",Wall Street analysts think Microsoft and Doceb...,https://www.fool.com/investing/2024/03/14/forg...,[MSFT.US],[],"{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}"
99,2024-03-14T00:00:00+00:00,"\nAnders Bylund | Mar 14, 2024\n","\nAnders Bylund | Mar 14, 2024\n",https://www.fool.com/terms/t/turnaround/,"[AAPL.US, MSFT.US]",[],"{'polarity': 0, 'neg': 0, 'neu': 1, 'pos': 0}"


In [56]:
#텍스트 데이터 전처리
def clean_text(text):
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text.strip()

# 전체 열에 전처리 함수 적용
df['content'] = df['content'].apply(clean_text)

In [57]:
#ChatOpenAI 객체 호출할때 파라미터 지정
llm = ChatOpenAI(model = "gpt-4",
                 openai_api_key = 'sk-', 
                 temperature = 0)

In [58]:
template = """
Identify the sentiment towards the MSFT stocks from the news article , where the sentiment score should be from -10 to +10 where -10 being the most negative and +10 being the most positve , and 0 being neutral
Also give the proper explanation for your answers and how would it effect the prices of different stocks
Article : {statement}
"""

#Langchain PromptTemplate 기능을 사용하여 프롬프트 생성
prompt = PromptTemplate(template = template, input_variables = ["statement"])
llm_chain = LLMChain(prompt = prompt, llm = llm)

In [59]:
print(llm_chain.run(df['content'][13]))

Sentiment Score: +8

Explanation: The sentiment in this article is highly positive towards Microsoft (MSFT) stocks. The article announces that Mustafa Suleyman, a well-known figure in the AI industry, is joining Microsoft to lead their newly created AI division. This is seen as a significant move that solidifies Microsoft's position as a leader in the AI sector. The article also mentions that several members of the Inflection AI team will be joining Microsoft, which could potentially bring more expertise and innovation to the company. 

The hiring of a high-profile figure like Suleyman and the addition of other AI experts to the team could potentially lead to the development of new, innovative products and services, which could increase the company's revenue and profitability in the long run. This could have a positive impact on MSFT stock prices. 

However, it's important to note that while this news is positive, the actual impact on stock prices will also depend on other factors such